In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf

# Enable mixed precision training
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

2025-07-18 04:03:19.492604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752811399.711220      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752811399.789986      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Paths
base_path = "/kaggle/input/dataset-for-crop-pest-and-disease-detection/Dataset for Crop Pest and Disease Detection"
train_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Cashew/train_set")
test_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Cashew/test_set")

# Parameters
img_size = (224, 224)
batch_size = 64
epochs = 35

# Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=35,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data Loaders
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Model
base_model = EfficientNetB3(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(768, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.4)(x)  # Increased dropout
x = BatchNormalization()(x)
output = Dense(train_data.num_classes, activation='softmax', dtype='float32')(x)
model = Model(inputs=base_model.input, outputs=output)

# Freeze base model initially
base_model.trainable = False

# Learning rate schedule
def lr_schedule(epoch):
    if epoch < 15:
        return 3e-4
    elif epoch < 25:
        return 1e-4
    else:
        return 5e-5

model.compile(
    optimizer=Adam(3e-4),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# Callbacks
checkpoint = ModelCheckpoint(
    "cashew_best.weights.h5",
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True
)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
lr_scheduler = LearningRateScheduler(lr_schedule)

# Phase 1: Frozen base model
history1 = model.fit(
    train_data,
    epochs=15,
    validation_data=test_data,
    callbacks=[checkpoint, early_stop, reduce_lr, lr_scheduler],
    verbose=1
)

# Phase 2: Unfreeze some layers
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False
    
model.compile(
    optimizer=Adam(1e-4),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

history2 = model.fit(
    train_data,
    initial_epoch=15,
    epochs=epochs,
    validation_data=test_data,
    callbacks=[checkpoint, early_stop, reduce_lr, lr_scheduler],
    verbose=1
)

# Evaluation
model.load_weights("cashew_best.weights.h5")
loss, acc = model.evaluate(test_data, verbose=0)
print(f"\n✅ Cashew Test Accuracy: {acc*100:.2f}%")

# Classification Report
Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)
print("\nClassification Report:")
print(classification_report(test_data.classes, y_pred, target_names=list(test_data.class_indices.keys())))

Found 18910 images belonging to 5 classes.
Found 6901 images belonging to 5 classes.


I0000 00:00:1752811426.085778      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1752811460.691360      72 service.cc:148] XLA service 0x7a107c0031d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752811460.692272      72 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752811464.091873      72 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/296 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - accuracy: 0.1875 - loss: 2.4978   

I0000 00:00:1752811480.981640      72 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


296/296 ━━━━━━━━━━━━━━━━━━━━ 540s 2s/step - accuracy: 0.2523 - loss: 2.0535 - val_accuracy: 0.1942 - val_loss: 1.6900 - learning_rate: 3.0000e-04
Epoch 2/15
296/296 ━━━━━━━━━━━━━━━━━━━━ 303s 1s/step - accuracy: 0.2873 - loss: 1.8554 - val_accuracy: 0.2439 - val_loss: 1.6694 - learning_rate: 3.0000e-04
Epoch 3/15
296/296 ━━━━━━━━━━━━━━━━━━━━ 294s 994ms/step - accuracy: 0.3100 - loss: 1.7793 - val_accuracy: 0.2058 - val_loss: 1.6624 - learning_rate: 3.0000e-04
Epoch 4/15
296/296 ━━━━━━━━━━━━━━━━━━━━ 295s 998ms/step - accuracy: 0.3113 - loss: 1.7447 - val_accuracy: 0.2656 - val_loss: 1.6433 - learning_rate: 3.0000e-04
Epoch 5/15
296/296 ━━━━━━━━━━━━━━━━━━━━ 295s 996ms/step - accuracy: 0.3197 - loss: 1.7015 - val_accuracy: 0.2524 - val_loss: 1.6373 - learning_rate: 3.0000e-04
Epoch 6/15
296/296 ━━━━━━━━━━━━━━━━━━━━ 303s 1s/step - accuracy: 0.3181 - loss: 1.6751 - val_accuracy: 0.2410 - val_loss: 1.6426 - learning_rate: 3.0000e-04
Epoch 7/15
296/296 ━━━━━━━━━━━━━━━━━━━━ 324s 1s/step - accur

In [3]:
# Paths
base_path = "/kaggle/input/dataset-for-crop-pest-and-disease-detection/Dataset for Crop Pest and Disease Detection"
train_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Cassava/train_set")
test_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Cassava/test_set")

# Handle duplicate classes
valid_classes = [c for c in os.listdir(train_dir) if c in os.listdir(test_dir)]

# Parameters
img_size = (224, 224)
batch_size = 64
epochs = 40

# Advanced Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=30,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data Loaders
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=valid_classes
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    classes=valid_classes
)

# Model with Stochastic Depth
base_model = EfficientNetB3(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(train_data.num_classes, activation='softmax', dtype='float32')(x)
model = Model(inputs=base_model.input, outputs=output)
# Class Weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_data.classes), y=train_data.classes)
class_weights = dict(enumerate(class_weights))

# Focal Loss
def focal_loss(gamma=2.0, alpha=0.25):
    def loss_fn(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        loss = alpha * tf.pow(1. - y_pred, gamma) * cross_entropy
        return tf.reduce_mean(loss)
    return loss_fn

model.compile(
    optimizer=Adam(2e-4),
    loss=focal_loss(),
    metrics=['accuracy']
)

# Callbacks
checkpoint = ModelCheckpoint(
    "cassava_best.weights.h5",
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True
)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

# Training
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=test_data,
    callbacks=[checkpoint, early_stop, reduce_lr],
    class_weight=class_weights,
    verbose=1
)

# Evaluation
model.load_weights("cassava_best.weights.h5")
loss, acc = model.evaluate(test_data, verbose=0)
print(f"\n✅ Cassava Test Accuracy: {acc*100:.2f}%")

# Classification Report
Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)
print("\nClassification Report:")
print(classification_report(test_data.classes, y_pred, target_names=list(test_data.class_indices.keys())))

Found 16971 images belonging to 5 classes.
Found 7510 images belonging to 5 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 668s 2s/step - accuracy: 0.5740 - loss: 0.1622 - val_accuracy: 0.1723 - val_loss: 0.1659 - learning_rate: 2.0000e-04
Epoch 2/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 284s 1s/step - accuracy: 0.8127 - loss: 0.1179 - val_accuracy: 0.4625 - val_loss: 0.1319 - learning_rate: 2.0000e-04
Epoch 3/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 282s 1s/step - accuracy: 0.8676 - loss: 0.0955 - val_accuracy: 0.6555 - val_loss: 0.1008 - learning_rate: 2.0000e-04
Epoch 4/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 277s 1s/step - accuracy: 0.8830 - loss: 0.0766 - val_accuracy: 0.7075 - val_loss: 0.0784 - learning_rate: 2.0000e-04
Epoch 5/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 283s 1s/step - accuracy: 0.8986 - loss: 0.0606 - val_accuracy: 0.6702 - val_loss: 0.0704 - learning_rate: 2.0000e-04
Epoch 6/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 282s 1s/step - accuracy: 0.9113 - loss: 0.0463 - val_accuracy: 0.1928 - val_loss: nan - learning_rate: 2.0000e-04
Epoch 7/40


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/reduce_lr_on_plateau.py:94: RuntimeWarning: invalid value encountered in less
  self.monitor_op = lambda a, b: np.less(a, b - self.min_delta)


266/266 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.9162 - loss: 0.0350 - val_accuracy: 0.8373 - val_loss: 0.0333 - learning_rate: 2.0000e-04
Epoch 8/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 297s 1s/step - accuracy: 0.9260 - loss: 0.0260 - val_accuracy: 0.7514 - val_loss: 0.0345 - learning_rate: 2.0000e-04
Epoch 9/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 282s 1s/step - accuracy: 0.9270 - loss: 0.0193 - val_accuracy: 0.5586 - val_loss: 0.0464 - learning_rate: 2.0000e-04
Epoch 10/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 294s 1s/step - accuracy: 0.9356 - loss: 0.0146 - val_accuracy: 0.4185 - val_loss: 0.0493 - learning_rate: 2.0000e-04
Epoch 11/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 300s 1s/step - accuracy: 0.9240 - loss: 0.0126 - val_accuracy: 0.2591 - val_loss: 0.0628 - learning_rate: 2.0000e-04
Epoch 12/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 299s 1s/step - accuracy: 0.9413 - loss: 0.0092 - val_accuracy: 0.8746 - val_loss: 0.0135 - learning_rate: 1.0000e-04
Epoch 13/40
266/266 ━━━━━━━━━━━━━━━━━━━━ 306s 1s/step - accuracy: 